In [1]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder

In [2]:
import xgboost

In [3]:
from xgboost import XGBClassifier

In [4]:
xgboost.__version__

'1.7.4'

## G3 predictions

In [5]:
data = pd.read_csv('../Field_training_data_noOutliers_newContaminationMetric.csv')
labels = pd.read_csv('../Field_training_labels_noOutliers_newContaminationMetric.csv')
data = data.reset_index(drop=True)
labels = labels.reset_index(drop=True)

idx = labels.index[labels['Trophic mode'] == 'Un']
train_labels = labels.drop(idx)
train_data = data.drop(idx)

features = pd.read_csv('../Extracted_Pfams_noOutliers_newContaminationMetric_xg.csv')

G3Diel_TPM_merged_lats = pd.read_csv('../../g3Diel/g3Diel_surface_tpm_updatedMarferret_marmicroDb2023_noOutliers_fixedTPM_fall2023.csv')

,PF00001,PF00002,PF00003,PF00004,PF00005,PF00006,PF00007,PF00008,PF00009,PF00010,...,PF19842,PF19974,PF20030,PF20043,PF20071,PF20102,PF20133,PF20143,PF20144,PF20145
0,0.000000,0.0,333.443824,1781.768026,3712.994806,315.882974,0.0,0.000000,1141.465482,0.000000,...,0.0,0.000000,0.000000,0.000000,1071.304959,0.000000,121.127164,91.579322,0.0,30.219007
1,0.000000,0.0,0.000000,4267.638751,7495.398350,5320.466540,0.0,0.000000,5951.765853,773.174526,...,0.0,0.000000,104.385977,0.000000,0.000000,0.000000,0.000000,80.189423,0.0,0.000000
2,24.594629,0.0,0.000000,2157.321818,1636.620519,394.249544,0.0,0.000000,646.218877,263.333745,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,0.000000,0.0,0.000000,1394.473767,3287.173648,1502.489224,0.0,0.446961,1901.018552,0.000000,...,0.0,14.946402,4.944797,6.108618,0.000000,0.000000,2.640483,28.681549,0.0,0.000000
4,4.759697,0.0,8.538634,3434.303467,1840.528624,2355.420955,0.0,153.538439,14475.798812,0.000000,...,0.0,0.000000,8.772830,0.000000,1810.393905,9.796741,1.416444,0.000000,0.0,45.478368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,0.000000,0.0,0.000000,258.822260,619.352215,0.000000,0.0,12.127863,36.210302,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,14.972606,0.000000,0.000000,0.0,0.000000
425,0.000000,0.0,26.870275,1182.377466,938.314802,130.498131,0.0,199.762824,2288.672304,699.653781,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,497.918243
426,0.000000,0.0,69.214160,4775.030013,4656.786667,1822.945605,0.0,0.000000,2144.258846,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,129.438131,0.000000,0.0,0.000000
427,0.000000,0.0,12.385237,5138.972115,1735.011574,5908.833880,0.0,41.782007,4781.347127,35.706823,...,0.0,0.000000,0.000000,0.000000,0.000000,137.766230,154.801018,6.314319,0.0,751.857625


In [6]:
G3Diel_TPM_merged_lats.columns[(G3Diel_TPM_merged_lats == 0).all()]

Index(['PF08713', 'PF14259'], dtype='object')

In [8]:
features = features['pfam']

In [15]:
len(features)

183

In [16]:
G3Diel_TPM_merged_lats = G3Diel_TPM_merged_lats[features]

In [18]:
G3Diel_TPM_merged_lats.columns[(G3Diel_TPM_merged_lats == 0).all()]

Index(['PF14259', 'PF08713'], dtype='object')

In [19]:
train_data = train_data[features]

In [20]:
le = LabelEncoder()

In [21]:
X,y = train_data, le.fit_transform(train_labels['Trophic mode'])
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [22]:
G3Diel = scaler.transform(G3Diel_TPM_merged_lats)

In [23]:
#gamma: 0.5, learning_rate: 0.2, max_depth: 3, n_estimators: 100, reg_lambda: 0.0 
model = XGBClassifier(gamma=0.5, learning_rate=0.2, max_depth=3, n_estimators=100, reg_lambda=0.0)
model.fit(X,y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.5, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [24]:
xg_predictions_G3Diel = model.predict(G3Diel)

In [26]:
prob = model.predict_proba(G3Diel)

In [27]:
G3Diel_predictions = pd.DataFrame(data={'xg_pred':xg_predictions_G3Diel})

In [28]:
probDF = pd.DataFrame(data={'probability':np.max(prob, axis = 1)})

In [30]:
G3Diel_predictions.to_csv('../../g3Diel/G3_diel_trophicModePredictions_updatedMarferret_marmicroDb2023_noOutliers_fall2023',index=False)

In [31]:
probDF.to_csv('../../g3Diel/G3_diel_trophicModePredictionsProbabilities_updatedMarferret_marmicroDb2023_noOutliers_fall2023',index=False)